# Simple example for concurrent processing of coroutines with `asyncio`

In [1]:
import asyncio
import datetime
import time
import contextlib

In [2]:
def now():
    return datetime.datetime.now().time()

## Define a simple context manager for measuring the execution time of statements inside its `with` block

In [3]:
@contextlib.contextmanager
def measure_time():
    start = time.perf_counter()
    yield
    end = time.perf_counter()
    print(f"Needed {end - start} seconds.")

## Create a simple coroutine that waits for some time before returning its result
Note that this coroutine simply sleeps and returns control to the event loop. Instead of `await`ing `asyncio.sleep(...)`, one could `await` any kind of I/O operation:
* HTTP requests and other network activity
* file system operations
* executing database queries
* ...

In [4]:
async def wait(seconds):
    print(f"{now()} begin {seconds}")
    await asyncio.sleep(seconds)
    print(f"{now()} end   {seconds}")
    return seconds

## Multiple sequential `await` statements
Note that the runtime is the sum of the runtimes of the coroutines.

In [5]:
with measure_time():
    print(await wait(0.5))
    print(await wait(0.8))
    print(await wait(0.3))    

16:00:09.743974 begin 0.5
16:00:10.244732 end   0.5
0.5
16:00:10.244892 begin 0.8
16:00:11.045921 end   0.8
0.8
16:00:11.046092 begin 0.3
16:00:11.346967 end   0.3
0.3
Needed 1.603213521069847 seconds.


## Parallelization with `asyncio.gather`
Two coroutines can be sent to the event loop in parallel, such that the longest running coroutine determines the total runtime.

While one coroutine is `await`ing results, other coroutines can be scheduled by the event loop and do useful work!

In [6]:
with measure_time():
    print(await asyncio.gather(wait(0.5), wait(0.8), wait(0.3)))

16:00:11.358326 begin 0.5
16:00:11.358462 begin 0.8
16:00:11.358539 begin 0.3
16:00:11.659095 end   0.3
16:00:11.859722 end   0.5
16:00:12.159352 end   0.8
[0.5, 0.8, 0.3]
Needed 0.8013974509667605 seconds.


## Comparison with synchronous waiting
The final result is the same if we use synchronous waiting, i.e., `time.sleep(...)` instead of `asyncio.sleep(...)`. However, the event loop is blocked while `time.sleep()` runs, such that `asyncio.gather(...)` does not result in a concurrent execution of the coroutines, and the total runtime is the sum of the inividual coroutine runtimes:

In [8]:
async def wait_sync(seconds):
    print(f"{now()} begin {seconds}")
    time.sleep(seconds)
    print(f"{now()} end   {seconds}")
    return seconds

with measure_time():
    print(await asyncio.gather(wait_sync(0.5), wait_sync(0.8), wait_sync(0.3)))

16:00:29.676472 begin 0.5
16:00:30.177207 end   0.5
16:00:30.177419 begin 0.8
16:00:30.978312 end   0.8
16:00:30.978583 begin 0.3
16:00:31.278985 end   0.3
[0.5, 0.8, 0.3]
Needed 1.6029145860811695 seconds.


## Generating the coroutines which are passed to `asyncio.gather`
It is often inconvenient to pass each individual coroutine to `asyncio.gather` explicitly. The coroutines can be created by a generator expression like this:

In [9]:
with measure_time():
    print(await asyncio.gather(*(wait(x) for x in (0.5, 0.8, 0.3))))

16:02:24.384242 begin 0.5
16:02:24.384361 begin 0.8
16:02:24.384414 begin 0.3
16:02:24.684938 end   0.3
16:02:24.885527 end   0.5
16:02:25.185712 end   0.8
[0.5, 0.8, 0.3]
Needed 0.8017592410324141 seconds.
